<a href="https://colab.research.google.com/github/Tiasangga19/Sentiment-transformer_kelompok/blob/main/TUGAS_4KELOMPOK_DL_LAJUT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install transformers accelerate datasets pandas scikit-learn torch

In [14]:
import pandas as pd

df = pd.read_csv("train.csv")
print(df["label"].value_counts())

label
2    100
1    100
0    100
Name: count, dtype: int64


In [7]:
import pandas as pd
import random

# Membuat data dummy agar file 'dataset_1.csv' tersedia
print("Sedang membuat file dummy dataset_1.csv...")

data = []
# Buat 600 baris data dummy (Cukup untuk dibagi 100/100/100 train + sisa test)
labels = [0, 1, 2]
texts = [
    "Makanan di sini enak sekali",
    "Pelayanannya sangat lambat",
    "Harganya standar saja",
    "Sangat mengecewakan",
    "Luar biasa bagus",
    "Biasa saja tidak ada yang spesial"
]

for i in range(1000):
    t = random.choice(texts) + f" {i}"
    l = random.choice(labels)
    data.append([t, l])

# Simpan sebagai dataset_1.csv
df_dummy = pd.DataFrame(data, columns=["Komentar", "Sentimen"])
df_dummy.to_csv("dataset_1.csv", index=False)

print("BERHASIL! File 'dataset_1.csv' sudah dibuat.")
print("Silakan jalankan ulang kode Anda yang error tadi.")

Sedang membuat file dummy dataset_1.csv...
BERHASIL! File 'dataset_1.csv' sudah dibuat.
Silakan jalankan ulang kode Anda yang error tadi.


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 1. Baca dataset yang kamu upload
# Pastikan nama file sesuai: 'dataset_1.csv'
df = pd.read_csv("dataset_1.csv")

# Ubah nama kolom agar standar (Komentar -> text, Sentimen -> label)
df = df.rename(columns={'Komentar': 'text', 'Sentimen': 'label'})

# 2. Pisahkan data berdasarkan label (Asumsi: 0=Negatif, 1=Netral, 2=Positif)
df_0 = df[df['label'] == 0]
df_1 = df[df['label'] == 1]
df_2 = df[df['label'] == 2]

# 3. Ambil 100 data dari masing-masing label untuk Training
train_0 = df_0.sample(n=100, random_state=42)
train_1 = df_1.sample(n=100, random_state=42)
train_2 = df_2.sample(n=100, random_state=42)

# Gabungkan menjadi train.csv (Total 300)
df_train = pd.concat([train_0, train_1, train_2]).sample(frac=1, random_state=42).reset_index(drop=True)
df_train.to_csv("train.csv", index=False)

# 4. Ambil 400 data sisa untuk Testing (Tanpa Label)
# Kita ambil dari sisa data yang tidak masuk ke training
sisa_data = df.drop(df_train.index)
df_test = sisa_data.sample(n=400, random_state=42)[['text']].reset_index(drop=True)
df_test.to_csv("test.csv", index=False)

print(f"Berhasil membuat 'train.csv' dengan {len(df_train)} baris (100 per kelas).")
print(f"Berhasil membuat 'test.csv' dengan {len(df_test)} baris (tanpa label).")
print("Contoh data train:\n", df_train['label'].value_counts())

Berhasil membuat 'train.csv' dengan 300 baris (100 per kelas).
Berhasil membuat 'test.csv' dengan 400 baris (tanpa label).
Contoh data train:
 label
2    100
1    100
0    100
Name: count, dtype: int64


In [16]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)

# Load data
df = pd.read_csv("train.csv")
dataset = Dataset.from_pandas(df)

# Load Transformer
MODEL_NAME = "indobenchmark/indobert-base-p1"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3
)

def tokenize(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

dataset = dataset.map(tokenize, batched=True)
dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "label"]
)

# Training Arguments (DIKUATKAN)
args = TrainingArguments(
    output_dir="./model",
    num_train_epochs=10,
    per_device_train_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=5,
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset
)

trainer.train()

model.save_pretrained("model")
tokenizer.save_pretrained("model")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Step,Training Loss
5,1.101800
10,1.181800
15,1.150600
20,1.207900
25,1.125300
30,1.143200
35,1.156500
40,1.154700
45,1.167500
50,1.131000


('model/tokenizer_config.json',
 'model/special_tokens_map.json',
 'model/vocab.txt',
 'model/added_tokens.json',
 'model/tokenizer.json')

In [12]:
# Load Model (num_labels=3 karena ada 3 kelas: 0, 1, 2)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=3)

# Setup Training
training_args = TrainingArguments(
    output_dir="./hasil_model",
    num_train_epochs=3,              # 3 Epoch
    per_device_train_batch_size=8,   # Batch size 8
    logging_steps=10,
    learning_rate=2e-5,
    save_strategy="no",
    report_to="none"
)

# Mulai Training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train,
)

print("Mulai training...")
trainer.train()
print("Training selesai!")

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/indobert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Mulai training...


Step,Training Loss
10,1.134700
20,1.129100
30,1.101100
40,1.131300
50,1.152100
60,1.115400
70,1.111900
80,1.106200
90,1.114300
100,1.105600


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Training selesai!


In [13]:
import numpy as np

# 1. Prediksi
print("Sedang memprediksi 400 data uji...")
predictions = trainer.predict(encoded_test)
preds = np.argmax(predictions.predictions, axis=-1)

# 2. Load test.csv asli untuk ditempel hasil
df_hasil = pd.read_csv("test.csv")

# 3. Mapping angka kembali ke kata (Sesuaikan jika terbalik)
# Asumsi di dataset ini: 0=Negatif, 1=Netral, 2=Positif (Cek ulang jika perlu)
label_map = {0: "Negatif", 1: "Netral", 2: "Positif"}
df_hasil['prediksi_label'] = [label_map[p] for p in preds]

# 4. Simpan ke CSV
nama_file_output = "tugas_kelompok_hasil_prediksi.csv"
df_hasil.to_csv(nama_file_output, index=False)

print(f"Sukses! File '{nama_file_output}' sudah jadi.")
print(df_hasil.head(10))

Sedang memprediksi 400 data uji...


Sukses! File 'tugas_kelompok_hasil_prediksi.csv' sudah jadi.
                                    text prediksi_label
0                   Luar biasa bagus 458        Negatif
1        Makanan di sini enak sekali 800        Negatif
2         Pelayanannya sangat lambat 696        Negatif
3              Harganya standar saja 455        Negatif
4  Biasa saja tidak ada yang spesial 621        Negatif
5                   Luar biasa bagus 512        Negatif
6        Makanan di sini enak sekali 534        Negatif
7                   Luar biasa bagus 589        Negatif
8              Harganya standar saja 600        Negatif
9        Makanan di sini enak sekali 656        Negatif


In [18]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax

# Load model
tokenizer = AutoTokenizer.from_pretrained("model")
model = AutoModelForSequenceClassification.from_pretrained("model")
model.eval()

test_df = pd.read_csv("test.csv")
labels = []
confidences = []

for text in test_df["text"]:
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128
    )

    with torch.no_grad():
        output = model(**inputs)

    probs = softmax(output.logits, dim=1)
    confidence, label = torch.max(probs, dim=1)

    labels.append(label.item())
    confidences.append(confidence.item())

label_map = {0: "negatif", 1: "netral", 2: "positif"}

test_df["label_prediksi"] = labels
test_df["sentimen"] = test_df["label_prediksi"].map(label_map)
test_df["confidence"] = confidences

test_df.to_csv("hasil_pelabelan.csv", index=False)


In [19]:
print(test_df["sentimen"].value_counts())


sentimen
netral     183
negatif    117
positif    100
Name: count, dtype: int64
